In [64]:
# ✅ Manipulación de Datos
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# ✅ TensorFlow y Keras
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# ✅ Visualización
import os
import matplotlib.pyplot as plt


In [65]:
# ✅ Rutas de los archivos
ruta_train_csv = r'C:\Users\mikel\OneDrive\Documentos\TB-DS-BIO-23.09.24\REPOSITORIOS\Mikel\clip_count\train.csv'
ruta_train_img = r'C:\Users\mikel\OneDrive\Documentos\TB-DS-BIO-23.09.24\REPOSITORIOS\Mikel\clip_count\train'
ruta_test_csv = r'C:\Users\mikel\OneDrive\Documentos\TB-DS-BIO-23.09.24\REPOSITORIOS\Mikel\clip_count\test.csv'
ruta_test_img = r'C:\Users\mikel\OneDrive\Documentos\TB-DS-BIO-23.09.24\REPOSITORIOS\Mikel\clip_count\test'
ruta_modelo = r'C:\Users\mikel\OneDrive\Documentos\TB-DS-BIO-23.09.24\REPOSITORIOS\Mikel\dsft2409_mikeltelo\PROJECTS\kaggle_clip_counting\modelo_final\modelo_clip_count_p.h5'
ruta_resultados = r'C:\Users\mikel\OneDrive\Documentos\TB-DS-BIO-23.09.24\REPOSITORIOS\Mikel\dsft2409_mikeltelo\PROJECTS\kaggle_clip_counting\submission_p_final.csv'


In [66]:
# ✅ Cargar archivos CSV
df_train = pd.read_csv(ruta_train_csv)
df_test = pd.read_csv(ruta_test_csv)

# ✅ Validar columnas
assert 'id' in df_train.columns, "❌ 'train.csv' debe tener una columna llamada 'id'."
assert 'clip_count' in df_train.columns, "❌ 'train.csv' debe tener una columna llamada 'clip_count'."
assert 'id' in df_test.columns, "❌ 'test.csv' debe tener una columna llamada 'id'."

# ✅ Eliminar valores nulos
df_train = df_train.dropna(subset=['id', 'clip_count'])
df_test = df_test.dropna(subset=['id'])

print("✅ CSVs cargados y validados correctamente.")


✅ CSVs cargados y validados correctamente.


In [67]:
# ✅ Filtrar imágenes existentes
def filtrar_ids_con_imagenes(df, ruta_img):
    ids_validos = []
    for img_id in df['id']:
        img_path = os.path.join(ruta_img, f"clips-{img_id}.png")
        if os.path.exists(img_path):
            ids_validos.append(img_id)
        else:
            print(f"❌ Imagen no encontrada: {img_path}")
    return df[df['id'].isin(ids_validos)]

df_train = filtrar_ids_con_imagenes(df_train, ruta_train_img)
df_test = filtrar_ids_con_imagenes(df_test, ruta_test_img)

print("✅ IDs validados con imágenes existentes.")


✅ IDs validados con imágenes existentes.


In [68]:
# ✅ Parámetros de imágenes
IMG_SIZE = (128, 128)
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE

# ✅ Función para cargar datos
def cargar_datos(df, ruta_img, objetivo=None):
    ids = df['id'].values
    objetivos = df[objetivo].values if objetivo else None
    
    def preprocesar_imagen(id, objetivo=None):
        img_path = tf.strings.join([ruta_img, "/clips-", tf.strings.as_string(id), ".png"])
        img = tf.io.read_file(img_path)
        img = tf.image.decode_png(img, channels=1)
        img = tf.image.resize(img, IMG_SIZE)
        img = img / 255.0
        
        if objetivo is not None:
            return img, objetivo
        return img
    
    if objetivo is not None:
        dataset = tf.data.Dataset.from_tensor_slices((ids, objetivos))
        dataset = dataset.map(lambda id, objetivo: preprocesar_imagen(id, objetivo), num_parallel_calls=AUTOTUNE)
    else:
        dataset = tf.data.Dataset.from_tensor_slices(ids)
        dataset = dataset.map(lambda id: preprocesar_imagen(id), num_parallel_calls=AUTOTUNE)
    
    dataset = dataset.batch(BATCH_SIZE).prefetch(AUTOTUNE)
    return dataset

# ✅ Dividir datos
df_train, df_val = train_test_split(df_train, test_size=0.2, random_state=42)

# ✅ Crear datasets
dataset_train = cargar_datos(df_train, ruta_train_img, objetivo='clip_count')
dataset_val = cargar_datos(df_val, ruta_train_img, objetivo='clip_count')
dataset_test = cargar_datos(df_test, ruta_test_img)

print("✅ Datasets creados correctamente.")


✅ Datasets creados correctamente.


In [69]:
# ✅ Cargar modelo
modelo = load_model(ruta_modelo)

# ✅ Descongelar capas finales
for capa in modelo.layers[-10:]:
    capa.trainable = True

from tensorflow.keras.metrics import RootMeanSquaredError

# ✅ Compilar el modelo con la métrica correcta
modelo.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='mean_squared_error',
    metrics=[RootMeanSquaredError(name='rmse')]  # Corregido
)



In [70]:
# ✅ Callbacks
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

checkpoint = ModelCheckpoint(
    'modelo_clip_count_reentrenado_v2.keras',
    monitor='val_loss',
    save_best_only=True
)


In [63]:
# ✅ Entrenamiento
historial = modelo.fit(
    dataset_train,
    validation_data=dataset_val,
    epochs=15,
    callbacks=[early_stopping, checkpoint]
)


Epoch 1/15
375/375 ━━━━━━━━━━━━━━━━━━━━ 129s 338ms/step - loss: 77.9774 - rmse: 8.8300 - val_loss: 5.7521 - val_rmse: 2.3984
Epoch 2/15
375/375 ━━━━━━━━━━━━━━━━━━━━ 122s 325ms/step - loss: 74.3590 - rmse: 8.6221 - val_loss: 4.3148 - val_rmse: 2.0772
Epoch 3/15
375/375 ━━━━━━━━━━━━━━━━━━━━ 127s 338ms/step - loss: 75.8144 - rmse: 8.7056 - val_loss: 23.8159 - val_rmse: 4.8802
Epoch 4/15
375/375 ━━━━━━━━━━━━━━━━━━━━ 121s 323ms/step - loss: 74.7380 - rmse: 8.6443 - val_loss: 6.8686 - val_rmse: 2.6208
Epoch 5/15
375/375 ━━━━━━━━━━━━━━━━━━━━ 122s 324ms/step - loss: 75.9204 - rmse: 8.7125 - val_loss: 12.3586 - val_rmse: 3.5155
Epoch 6/15
375/375 ━━━━━━━━━━━━━━━━━━━━ 123s 329ms/step - loss: 75.6601 - rmse: 8.6970 - val_loss: 4.9230 - val_rmse: 2.2188
Epoch 7/15
375/375 ━━━━━━━━━━━━━━━━━━━━ 125s 333ms/step - loss: 72.1044 - rmse: 8.4879 - val_loss: 4.0348 - val_rmse: 2.0087
Epoch 8/15
375/375 ━━━━━━━━━━━━━━━━━━━━ 125s 333ms/step - loss: 74.1687 - rmse: 8.6106 - val_loss: 22.5625 - val_rmse: 4.75

In [ ]:
# ✅ Evaluar
resultados = modelo.evaluate(dataset_val)
print("✅ Resultados en validación:", resultados)


In [ ]:
# ✅ Generar predicciones
predicciones = modelo.predict(dataset_test).flatten()
df_test['clip_count'] = np.round(predicciones).astype(int)

# ✅ Guardar predicciones
df_test[['id', 'clip_count']].to_csv(ruta_resultados, index=False)
print(f"✅ Archivo de predicciones guardado en '{ruta_resultados}'")
